# Modèle combiné : 

In [2]:
#  Soumission Kaggle - Stacking

import pandas as pd
import numpy as np
from pathlib import Path
from lightgbm import LGBMRegressor
from lightgbm import early_stopping
import joblib

# Chargement des données
data_path = Path("C:/Users/Optimiste/Videos/Concours/Prédiction_prix_plaques_russes/data")

train = pd.read_csv(data_path / "train_preprocessed_v2.csv")
test = pd.read_csv(data_path / "test_preprocessed_v2.csv")
preprocessors = joblib.load(data_path / "preprocessors.joblib")


In [4]:
train_plate = pd.read_csv(data_path / "train_plate.csv")
train = pd.merge(train, train_plate[["id", "gov_importance_level"]], on="id", how="left")


test_plate = pd.read_csv(data_path / "test_plate.csv")
test = pd.merge(test, test_plate[["id", "gov_importance_level"]], on="id", how="left")


# 1. Calcul de fréquence d'apparition des lettres
letter_freq = train['letters'].value_counts(normalize=True)  # fréquence relative

# 2. Rareté = 1 - fréquence
train['letters_rarity'] = train['letters'].map(lambda x: 1 - letter_freq.get(x, 0))


# test
letter_freq = test['letters'].value_counts(normalize=True)  # fréquence relative
test['letters_rarity'] = test['letters'].map(lambda x: 1 - letter_freq.get(x, 0))


In [6]:
# Variables finales retenues (4-5 max)
features = [
    'prestige_score',    # Composite gov_importance + digits + rarity
    'region_encoded',    # Région encodée
    'is_ultra_premium',  # Combinaison premium_plate + magic_numbers
    'plate_age_days'     # Fraîcheur de l'annonce
]

# Création du prestige_score (feature composite)
train['prestige_score'] = (
    0.4 * train['gov_importance_level']/10 + 
    0.3 * train['digits']/999 +
    0.3 * (1 - train['letters_rarity'])
)

# Ultra-premium (simplifié)
train['is_ultra_premium'] = (
    (train['premium_plate'] == 1) & 
    (train['digits'].astype(str).str.match(r'(\d)\1{2}'))  # 111, 222, etc.
).astype(int)

### Stacking léger (3 modèles Max)

In [17]:
from sklearn.model_selection import KFold
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import TimeSeriesSplit


# Modèles de base
base_models = [
    ('lgbm', LGBMRegressor(
        num_leaves=31,
        n_estimators=300,
        verbosity=-1
    )),
    ('xgb', XGBRegressor(
        max_depth=5,
        n_estimators=250
    ))
]

# Meta-modèle
stack = StackingRegressor(
    estimators=base_models,
    final_estimator=RidgeCV(),
    cv=KFold(n_splits=3, shuffle=True, random_state=42)
)

# Entraînement
stack.fit(train[features], train['log_price'])

StackingRegressor(cv=KFold(n_splits=3, random_state=42, shuffle=True),
                  estimators=[('lgbm',
                               LGBMRegressor(n_estimators=300, verbosity=-1)),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feat...
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=5,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=250, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...))],
                  final_estimator=RidgeCV())

### Validation croisée Optimisée

In [22]:
from sklearn.metrics import mean_absolute_error
from sklearn.base import clone
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

# SMAPE personnalisé
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# Préparation
X = train[features]
y = train['log_price']
tscv = TimeSeriesSplit(n_splits=5)

oof_preds = np.zeros(len(train))

# Boucle manuelle de validation
for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    print(f"Fold {fold+1}")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    model = clone(stack)
    model.fit(X_train, y_train)
    oof_preds[val_idx] = model.predict(X_val)

# Inverse transformation
oof_preds_rub = np.expm1(oof_preds)
y_true_rub = np.expm1(y)

# Calcul SMAPE
score = smape(y_true_rub, oof_preds_rub)
print(f"\nSMAPE sur validation (TimeSeriesSplit): {score:.2f}%")


Fold 1
Fold 2
Fold 3
Fold 4
Fold 5

SMAPE sur validation (TimeSeriesSplit): 93.97%


### Pepiline complet avec peu de Features

In [25]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import QuantileTransformer

# Pipeline final
final_pipe = make_pipeline(
    QuantileTransformer(output_distribution='normal'),  # Gestion des outliers
    StackingRegressor(
        estimators=[
            ('lgbm', LGBMRegressor(verbose=-1)),
            ('xgb', XGBRegressor())
        ],
        final_estimator=RidgeCV()
    )
)

# Entraînement
final_pipe.fit(train[features], train['log_price'])

C:\Users\Optimiste\.conda\envs\kaggle_plates\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\Optimiste\.conda\envs\kaggle_plates\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\Optimiste\.conda\envs\kaggle_plates\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\Optimiste\.conda\envs\kaggle_plates\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\Optimiste\.conda\envs\kaggle_plates\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but L

Pipeline(steps=[('quantiletransformer',
                 QuantileTransformer(output_distribution='normal')),
                ('stackingregressor',
                 StackingRegressor(estimators=[('lgbm',
                                                LGBMRegressor(verbose=-1)),
                                               ('xgb',
                                                XGBRegressor(base_score=None,
                                                             booster=None,
                                                             callbacks=None,
                                                             colsample_bylevel=None,
                                                             colsample_bynode=None,
                                                             colsample_bytree=None,
                                                             device=None,
                                                             early_stopping_rounds=None,
                                                             enab...
                                                             importance_type=None,
                                                             interaction_constraints=None,
                                                             learning_rate=None,
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=None,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=None,
                                                             n_jobs=None,
                                                             num_parallel_tree=None,
                                                             random_state=None, ...))],
                                   final_estimator=RidgeCV()))])

### Génération des prédictions

In [32]:
import numpy as np
import pandas as pd
import joblib
from datetime import datetime
from pathlib import Path

# ===========================
# 🔁 Calcul des variables test
# ===========================

# Création des variables pour le test set
test['prestige_score'] = (
    0.4 * test['gov_importance_level'] / 10 +
    0.3 * test['digits'] / 999 +
    0.3 * (1 - test['letters_rarity'])
)

test['is_ultra_premium'] = (
    (test['premium_plate'] == 1) &
    (test['digits'].astype(str).str.match(r'(\d)\1{2}'))
).astype(int)

# ===========================
# 🔮 Prédictions finales
# ===========================

test_preds = np.expm1(final_pipe.predict(test[features]))  # prédiction log → réel

# Post-traitement : boost sur ultra-premium et clip minimum
final_preds = np.where(
    test['is_ultra_premium'] == 1,
    test_preds * 1.15,  # boost de 15 %
    test_preds
).clip(50000, None)  # minimum réaliste = 50k RUB

# ===========================
# 💾 Sauvegarde des fichiers
# ===========================

# Dossier de sauvegarde
model_dir = Path("C:/Users/Optimiste/Videos/Concours/Prédiction_prix_plaques_russes/model")
model_dir.mkdir(exist_ok=True)

# Nom unique basé sur l'horodatage
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
model_name = f"stacking_model_{timestamp}"  # ← tu peux changer pour lgbm_model_... si besoin

# 📁 Sauvegarde du modèle
joblib.dump(final_pipe, model_dir / f"{model_name}.joblib")

# 📁 Sauvegarde du fichier de soumission
submission = pd.DataFrame({'ID': test['id'], 'Price': final_preds.round()})
submission.to_csv(model_dir / f"submission_{model_name}.csv", index=False)

# 📁 Sauvegarde des métadonnées
with open(model_dir / f"metadata_{model_name}.txt", "w") as f:
    f.write(f"""Modèle StackingRegressor - Soumission Kaggle
Date: {timestamp}
Features utilisées: {features}
Statistiques des prédictions :
{submission['Price'].describe()}
""")


C:\Users\Optimiste\.conda\envs\kaggle_plates\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [40]:
# Sauvegarder la version enrichie du jeu d'entraînement et de test
data_path = Path("C:/Users/Optimiste/Videos/Concours/Prédiction_prix_plaques_russes/data")

train.to_csv(data_path /"train_2605.csv", index=False)
test.to_csv(data_path /"test_2605.csv", index=False)
